# Pointwise Regression

In this notebook, we examine the pointwise regression approach, including tuning. First, we load the required dependencies and the data.

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
from category_encoders.binary import BinaryEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from datetime import timedelta
from sklearn.compose import ColumnTransformer
from time import time

from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType
from src.pipeline.pipeline_transformers import *


# load the data
train_df = config.load_traindata_for_pointwise()
pipelineFactory = PipelineFactory()

## Pointwise Regression without Target Normalization

In [3]:
start = time()

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_REGRESSION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.CROSS_VALIDATION,
    n_folds=5,
    workers=1,
    target="rank"
)
pipeline.run()

runtime = int(time() - start)
print('\nruntime: ' + str(timedelta(seconds=runtime)) + ' [' + str(runtime) + 's]')

Creating pipeline ...
Starting pipeline using method: EvaluationType.CROSS_VALIDATION


100%|█████████████████████████████████████████████| 5/5 [01:00<00:00, 12.16s/it]

Finished running the pipeline
Evaluation metrics:
    validation_average_spearman_fold_0: 0.764
    validation_average_spearman_fold_1: 0.7641
    validation_average_spearman_fold_2: 0.7238
    validation_average_spearman_fold_3: 0.7812
    validation_average_spearman_fold_4: 0.775
    average of all folds: 0.7616 [std=0.02]

runtime: 0:01:13 [73s]


The performance is quite good with 0.76. Now, let us try this approach with target normalization which is supposed to improve performance.

## Pointwise Regression with Target Normalization

In [4]:
start = time()

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_NORMALIZED_REGRESSION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.CROSS_VALIDATION,
    n_folds=5,
    workers=1,
    target="rank"
)
pipeline.run()

runtime = int(time() - start)
print('\nruntime: ' + str(timedelta(seconds=runtime)) + ' [' + str(runtime) + 's]')

Creating pipeline ...
Starting pipeline using method: EvaluationType.CROSS_VALIDATION


100%|█████████████████████████████████████████████| 5/5 [01:10<00:00, 14.11s/it]

Finished running the pipeline
Evaluation metrics:
    validation_average_spearman_fold_0: 0.8044
    validation_average_spearman_fold_1: 0.8038
    validation_average_spearman_fold_2: 0.783
    validation_average_spearman_fold_3: 0.814
    validation_average_spearman_fold_4: 0.8114
    average of all folds: 0.8033 [std=0.0109]

runtime: 0:01:30 [90s]


In [5]:
start = time()

param_grid = {
    "encoder_transformer__batch_size": [50, 500],
}

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_NORMALIZED_REGRESSION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.GRID_SEARCH,
    param_grid=param_grid,
    n_folds=2,
    workers=1,
    target="rank"
)

pipeline.run()

runtime = int(time() - start)
print('\nruntime: ' + str(timedelta(seconds=runtime)) + ' [' + str(runtime) + 's]')

Creating pipeline ...
Starting pipeline using method: EvaluationType.GRID_SEARCH
Performing grid search


100%|█████████████████████████████████████████████| 2/2 [00:55<00:00, 27.61s/it]

Finished running the pipeline
Evaluation metrics:
    best_score: 0.806
    best_params: {'encoder_transformer__batch_size': 500}

runtime: 0:01:02 [62s]


### Tuning with Bayes Search

In [3]:
start = time()

# number of optimization rounds = n_iter / n_points (e.g. 50 rounds in our case)
n_iter = 200 # how many unique parameters to examine - our default: 200
n_points = 4 # how many unique parameter combinations per optimization round - our default: 4
cv = 4 # how many fits for each unique parameter combination - our default: 4
n_jobs = -1 # how many fits in parallel (only parallelizable per round) - our default: -1

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_NORMALIZED_REGRESSION_BAYES_SEARCH,
    verbose_level=1,
    target="rank",
    bayes_n_iter=n_iter,
    bayes_n_points=n_points,
    bayes_cv=cv,
    bayes_n_jobs=n_jobs
)

pipeline.run()

runtime = int(time() - start)
print('\nruntime: ' + str(timedelta(seconds=runtime)) + ' [' + str(runtime) + 's]')

Creating pipeline ...


c:\Users\patri\Documents\KIT\Data Science - Praktikum\Projekte\phase-2\src\pipeline\pipeline_transformers.py:519: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_new = pd.Series()


Starting pipeline using method: EvaluationType.BAYES_SEARCH
Performing bayes search
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds 

c:\Users\patri\anaconda3\envs\DS\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits


...

Fitting 4 folds for each of 4 candidates, totalling 16 fits


c:\Users\patri\anaconda3\envs\DS\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Fitting 4 folds for each of 4 candidates, totalling 16 fits
best score: 0.8020292900043039
best params:
    dataset_transformer__encoder: None
    dataset_transformer__expected_pca_variance: 1.0
    dataset_transformer__nan_ratio_feature_drop_threshold: 0.5
    encoder_transformer__encoder: OneHotEncoder()
    estimator__max_depth: 50
    estimator__max_features: None
    estimator__min_samples_leaf: 1
    estimator__min_samples_split: 5
    general_transformer__model_encoder: OneHotEncoder()
    general_transformer__scoring_encoder: TargetEncoder()
    general_transformer_

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\patri\Documents\KIT\Data Science - Praktikum\Projekte\phase-2\src\pipeline\pipeline_transformers.py:535: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_new = pd.Series()
c:\Users\patri\Documents\KIT\Data Science - Praktikum\Projekte\phase-2\src\pipeline\pipeline_transformers.py:535: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_new = pd.Series()
 20%|██        | 1/5 [00:21<01:27, 21.83s/it]c:\Users\patri\Documents\KIT\Data Science - Praktikum\Projekte\phase-2\src\pipeline\pipeline_transformers.py:535: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_new = pd.Series()
c:\Users\patri\Documents\KI

Finished running the pipeline
Evaluation metrics:
    validation_average_spearman_fold_0: 0.8232 [std=0.]
    validation_average_spearman_fold_1: 0.808 [std=0.]
    validation_average_spearman_fold_2: 0.7911 [std=0.]
    validation_average_spearman_fold_3: 0.8149 [std=0.]
    validation_average_spearman_fold_4: 0.8195 [std=0.]
    average_spearman (5-fold): 0.8113 [std=0.0113]

runtime: 2:20:56 [8456s]
